In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# 设置请求头部信息，模拟浏览器请求
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
# 设置YachtWorld网站上的搜索条件
search_params = {
    'class': 'sail',
    # 'price': 'USD100000-500000',
    # 'length': '40-50ft'
}
base_url = 'https://www.yachtworld.com'

requests.get(base_url, 
    headers=headers)

<Response [200]>

## 探索性分析每一个具体url的页面结构

In [2]:
#BoatDetails > div.content.nav-slide.with-sticky-contact.home-block > div.boat-details > div.body > div.boat-details-content > div.details
link = 'https://www.yachtworld.com/yacht/2009-royal-huisman-j-class-yacht-6750313/'
res = requests.get(link, headers=headers)
soup = BeautifulSoup(res.text, 'html.parser')
soup.text

'HANUMAN Racer Royal Huisman for sale - YachtWorld✕YachtWorld AppMake your dream come true todayOpenBoatsPower BoatsSailboatsSell Your BoatPersonal Boat ShopperResearchServicesYacht BrokersBoat DealersSurveyorsYacht LoansInsuranceBoat TransportDirectoryToggle NavigationUnited States (English) ▼ Danmark - Dansk Deutschland - Deutsch Australia - English United Kingdom - English España - Español Suomi - suomi France - Français Italia - Italiano Nederland - Nederlands Norge - Norsk Sverige - SvenskaLogin  CloseBurgess LondonCunard House, London, SW1Y 4LR, United KingdomView phone numberCloseContact the Seller* Please correct highlighted errors.NamePhone (optional)EmailMessageI recently viewed your listing on YachtWorld and I am interested in more details. Thank you.NameContact BrokerHome⁄/Boats for sale⁄/Sail⁄/Racer⁄/Royal Huisman⁄/2009 Royal Huisman J Class Yacht2009 Royal Huisman J Class YachtUS$14,900,000Newport, Rhode Island1/14Image coming soonImage coming soonImage coming soonImage c

In [3]:
detail = soup.select_one('#BoatDetails > div.content.nav-slide.with-sticky-contact.home-block > div.boat-details > div.body > div.boat-details-content > div.details')
detail.text

'BasicsYear2009MakeRoyal HuismanModelJ Class YachtClassRacerLength138.12ftFuel TypeDieselHull MaterialAluminumOffered ByBurgess LondonDescription2009 Royal Huisman J Class YachtBuilt in The Netherlands by Royal Huisman, HANUMAN is one of the most remarkable and highly revered J-Class sailing yachts. HANUMAN has benefited from a vast level of highly customised refits including a complete overhaul of all her racing and sailing systems in advance of the 2017 World Championships. She is "the" J-Class to own and offered for sale with a huge inventory. Her spectacular interior, created by Pieter Beeldsnijder, accommodates up to six guests in 3 en suite cabins, including a full beam owner\'s cabin.Show MoreContact InformationPlease contact Henry Craven-Smith at +44 (0)7796 278 797Other DetailsDisclaimerThe Company offers the details of this vessel in good faith but cannot guarantee or warrant the accuracy of this information nor warrant the condition of the vessel. A buyer should instruct his

In [4]:
items = detail.select('div.header')
len(items)
items[0].text
items[0].next_sibling
for i in range(len(items)):
    if items[i].text.lower()=='specifications':
        break
i
# items[i].text

6

In [5]:
def kv_attributes(basic, name='Basic'):
    titles = [f'{name}.{"".join(map(lambda x:x.capitalize(),t.text.strip().split()))}' for t in basic.select('td.datatable-title')]
    values = [t.text.strip() for t in basic.select('td.datatable-value')]
    return {k:v for k,v in zip(titles, values)}
kv_attributes(items[0].next_sibling)

{'Basic.Year': '2009',
 'Basic.Make': 'Royal Huisman',
 'Basic.Model': 'J Class Yacht',
 'Basic.Class': 'Racer',
 'Basic.Length': '138.12ft',
 'Basic.FuelType': 'Diesel',
 'Basic.HullMaterial': 'Aluminum',
 'Basic.OfferedBy': 'Burgess London'}

In [6]:
def specification_attributes(specification, name='Specification'):
    sub_categories = specification.select('div.datatable-category')
    titles = []
    values = []
    for category in sub_categories:
        cate_name = category.find(attrs='sub-title').text
        cate_name = ''.join(map(lambda x:x.capitalize(), cate_name.split()))
        titles += [f'{name}.{cate_name}.{"".join(map(lambda x:x.capitalize(),t.text.strip().split()))}' for t in category.select('td.datatable-title')]
        values += [t.text.strip() for t in category.select('td.datatable-value')]
    return {k:v for k,v in zip(titles, values)}
specification_attributes(items[4].next_sibling, 'Features')

{'Features.ElectricalEquipment.ShorePowerInlet': '✓',
 'Features.ElectricalEquipment.Generator': '✓',
 'Features.Electronics.Depthsounder': '✓',
 'Features.Electronics.Radar': '✓',
 'Features.Electronics.Log-speedometer': '✓',
 'Features.Electronics.RadarDetector': '✓',
 'Features.Electronics.WindSpeedAndDirection': '✓',
 'Features.Electronics.Computer': '✓',
 'Features.Electronics.Repeater(s)': '✓',
 'Features.Electronics.TvSet': '✓',
 'Features.Electronics.NavigationCenter': '✓',
 'Features.Electronics.Plotter': '✓',
 'Features.Electronics.DvdPlayer': '✓',
 'Features.Electronics.Autopilot': '✓',
 'Features.Electronics.Radio': '✓',
 'Features.Electronics.Compas': '✓',
 'Features.Electronics.Gps': '✓',
 'Features.Electronics.CockpitSpeakers': '✓',
 'Features.Electronics.Vhf': '✓',
 'Features.InsideEquipment.Dishwasher': '✓',
 'Features.InsideEquipment.BowThruster': '✓',
 'Features.InsideEquipment.WashingMachine': '✓',
 'Features.InsideEquipment.ElectricBilgePump': '✓',
 'Features.Insid

In [7]:
def get_price(soup):

    price = soup.select_one('#BoatDetails > div.content.nav-slide.with-sticky-contact.home-block > div.boat-details > div.body > div > div.summary > div > span.payment-total')

    # price.text.strip().split('$')[1]
    x = float("".join(list(filter(str.isdigit,price.text.split('US$')[1]))))
    return x
get_price(soup)

14900000.0

In [8]:
sn = requests.Session()

In [9]:
# 爬取所有帆船的year属性和价格数据，并将它们存储到CSV文件中
import warnings


def get_yacht_data(url):
    # res = requests.get(url, headers=headers)
    res = sn.get(url, headers=headers)
    return get_yacht_data_req_res(res)
    
def get_yacht_data_req_res(req_res):
    soup = BeautifulSoup(req_res.text, 'html.parser')
    detail = soup.select_one('#BoatDetails > div.content.nav-slide.with-sticky-contact.home-block > div.boat-details > div.body > div.boat-details-content > div.details')
    assert detail is not None
    items = detail.select('div.header')
    assert len(items)>0
    result = {}
    for i, item in enumerate(items):
            if item.text.lower()=='BASICS'.lower() or item.text.lower()=='PROPULSION'.lower():
                result.update(kv_attributes(items[i].next_sibling, name=item.text.lower().capitalize()))
            elif item.text.lower()=='SPECIFICATIONS'.lower() or item.text.lower()=='FEATURES'.lower():
                result.update(specification_attributes(items[i].next_sibling, name=item.text.lower().capitalize()))
    try:
        result['Price'] = get_price(soup)
    except Exception as e:
        # warnings.warn(f"no price for {url}")
        pass
    return result
    
get_yacht_data(link)

{'Basics.Year': '2009',
 'Basics.Make': 'Royal Huisman',
 'Basics.Model': 'J Class Yacht',
 'Basics.Class': 'Racer',
 'Basics.Length': '138.12ft',
 'Basics.FuelType': 'Diesel',
 'Basics.HullMaterial': 'Aluminum',
 'Basics.OfferedBy': 'Burgess London',
 'Features.ElectricalEquipment.ShorePowerInlet': '✓',
 'Features.ElectricalEquipment.Generator': '✓',
 'Features.Electronics.Depthsounder': '✓',
 'Features.Electronics.Radar': '✓',
 'Features.Electronics.Log-speedometer': '✓',
 'Features.Electronics.RadarDetector': '✓',
 'Features.Electronics.WindSpeedAndDirection': '✓',
 'Features.Electronics.Computer': '✓',
 'Features.Electronics.Repeater(s)': '✓',
 'Features.Electronics.TvSet': '✓',
 'Features.Electronics.NavigationCenter': '✓',
 'Features.Electronics.Plotter': '✓',
 'Features.Electronics.DvdPlayer': '✓',
 'Features.Electronics.Autopilot': '✓',
 'Features.Electronics.Radio': '✓',
 'Features.Electronics.Compas': '✓',
 'Features.Electronics.Gps': '✓',
 'Features.Electronics.CockpitSpeake

In [10]:
with open('old_links.txt', 'r') as f:
    links = f.read().split()
links[:5]

['https://www.yachtworld.com/yacht/2021-sunreef-60-sail-8672222/',
 'https://www.yachtworld.com/yacht/1979-orion-sparkman-$-stephens-sloop-8474326/',
 'https://www.yachtworld.com/yacht/1981-baltic-42-dp-8381625/',
 'https://www.yachtworld.com/yacht/2023-jeanneau-sun-odyssey-349-7835504/',
 'https://www.yachtworld.com/yacht/2014-schock-harbor-25-8432704/']

In [11]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import numpy as np
from tqdm import tqdm
yacht_data_list = list(range(len(links)))
error_list = {}
def set_data(i):
    # print(i)
    try:
        yacht_data_list[i] = get_yacht_data(links[i])
    except Exception as e:
        time.sleep(60)
        try:
            yacht_data_list[i] = get_yacht_data(links[i])
        except:
            error_list[i]=e
            warnings.warn(f"error for {i}: {e}")
        else:
            error_list.pop(i, None)
    else:
        error_list.pop(i, None)

In [12]:
# with ThreadPoolExecutor(max_workers=64) as t:
with ThreadPoolExecutor(max_workers=32) as t:
    tasks = [t.submit(set_data, i) for i in range(len(links))]
    # tasks = [t.submit(set_data, i) for i in np.random.choice(len(links), 64, replace=False)]
    # tasks = [t.submit(set_data, i) for i in range(64)]
    for future in tqdm(as_completed(tasks), total=len(tasks)):
        pass    

 14%|█▍        | 1428/9975 [04:41<05:44, 24.83it/s]  /tmp/ipykernel_284707/1465481981.py:17: UserWarning: error for 1419: 
  warnings.warn(f"error for {i}: {e}")
 14%|█▍        | 1430/9975 [05:17<5:52:21,  2.47s/it]/tmp/ipykernel_284707/1465481981.py:17: UserWarning: error for 1424: 
  warnings.warn(f"error for {i}: {e}")
 14%|█▍        | 1431/9975 [05:18<5:35:45,  2.36s/it]/tmp/ipykernel_284707/1465481981.py:17: UserWarning: error for 1427: 
  warnings.warn(f"error for {i}: {e}")
/tmp/ipykernel_284707/1465481981.py:17: UserWarning: error for 1428: 
  warnings.warn(f"error for {i}: {e}")
/tmp/ipykernel_284707/1465481981.py:17: UserWarning: error for 1431: 
  warnings.warn(f"error for {i}: {e}")
/tmp/ipykernel_284707/1465481981.py:17: UserWarning: error for 1434: 
  warnings.warn(f"error for {i}: {e}")
 14%|█▍        | 1435/9975 [05:19<4:25:49,  1.87s/it]/tmp/ipykernel_284707/1465481981.py:17: UserWarning: error for 1435: 
  warnings.warn(f"error for {i}: {e}")
/tmp/ipykernel_284707/146

In [ ]:
a = {1:2}
a.pop(1)
a.pop(3, None)
a

{}

In [ ]:
error_list # , yacht_data_list
# error_list, yacht_data_list
# links[1890]
len(error_list)

575

In [21]:
with ThreadPoolExecutor(max_workers=32) as t:
    tasks = [t.submit(set_data, k) for k in error_list.keys()]
    # tasks = [t.submit(set_data, i) for i in np.random.choice(len(links), 64, replace=False)]
    # tasks = [t.submit(set_data, i) for i in range(64)]
    for future in tqdm(as_completed(tasks), total=len(tasks)):
        pass   

 75%|███████▍  | 431/575 [01:08<04:10,  1.74s/it]C:\Users\YeCanming\AppData\Local\Temp\ipykernel_23636\1465481981.py:17: UserWarning: error for 2912: 
  warnings.warn(f"error for {i}: {e}")
 98%|█████████▊| 565/575 [03:56<00:58,  5.84s/it]C:\Users\YeCanming\AppData\Local\Temp\ipykernel_23636\1465481981.py:17: UserWarning: error for 5933: 
  warnings.warn(f"error for {i}: {e}")
 99%|█████████▉| 569/575 [05:19<02:01, 20.23s/it]C:\Users\YeCanming\AppData\Local\Temp\ipykernel_23636\1465481981.py:17: UserWarning: error for 8208: 
  warnings.warn(f"error for {i}: {e}")
 99%|█████████▉| 570/575 [06:35<03:05, 37.05s/it]

In [17]:
links[1860], 
# get_yacht_data(links[2106])

('https://www.yachtworld.com/yacht/1999-bavaria-36-holiday-8171564/',)

In [24]:
data_list = list(filter(lambda x:not isinstance(x, int) and isinstance(x, dict), yacht_data_list))
len(data_list)

2152

In [26]:
df = pd.DataFrame(data_list)
df.head()

,Basic.Year,Basic.Make,Basic.Model,Basic.Class,Basic.Length,Basic.FuelType,Basic.HullMaterial,Basic.OfferedBy,name,Basic.EngineMake,...,Features.OutsideEquipment.WindGenerator,Features.InsideEquipment.Seakeeper,Features.InsideEquipment.AirCompressor,Features.InsideEquipment.ChemicalHead,Specifications.Miscellaneous.LiferaftCapacity,Features.Rigging.HydraulicWinch,Basic.RopeCutter,Specifications.Miscellaneous.DeadriseAtTransom,Features.AdditionalEquipment.LaunchingTrailer,Features.AdditionalEquipment.BeachingLegs
0,2023,Beneteau,Oceanis 46.1 - In Stock,Cruiser,46ft,Diesel,Fiberglass,"Cape Yachts at South Wharf - South Dartmouth, MA",Propulsion,yanmar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023,Jeanneau,Sun Odyssey 440,Cruiser,44ft,Diesel,Fiberglass,Marine Servicenter - San Diego,Propulsion,Yanmar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023,Jeanneau,Sun Odyssey 490,Cruiser,49ft,Diesel,Fiberglass,Marine Servicenter - Seattle,Propulsion,Yanmar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023,Jeanneau,60,Cruiser,60ft,Diesel,Fiberglass,Marine Servicenter - San Diego,Propulsion,Yanmar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023,Jeanneau,Yacht 65,Cruiser,65ft,Diesel,Fiberglass,Marine Servicenter - Seattle,Propulsion,Volvo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
with pd.ExcelWriter('../../data/爬取的数据.xlsx') as writer:
    df.to_excel(writer, sheet_name='二手帆船', index=False)